In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, MaxPooling1D, Flatten
from sklearn.preprocessing import MinMaxScaler

In [ ]:
############################### DATAFRAME 생성 ##################################
# 파일 경로 및 파일 개수 설정
file_prefix = "case_"
file_suffix = "_raw_data.csv"
num_files = 500

# 파일을 저장할 빈 리스트 생성
dfs = []

# 파일을 읽어 DataFrame으로 변환하여 리스트에 추가
for i in range(1, num_files + 1):
    file_path = f"{file_prefix}{i}{file_suffix}"
    
    # CSV 파일을 DataFrame으로 읽어오기
    df = pd.read_csv(file_path)
    
    # 리스트에 DataFrame 추가
    dfs.append(df)

# 'Value' column을 TensorFlow Dataset으로 변환
features =  [tf.convert_to_tensor(tf.constant(df['Value'], dtype=tf.float32)) for df in dfs]

# Feature normalization using MinMaxScaler
scaler = MinMaxScaler()
normalized_features = [scaler.fit_transform(feature.numpy().reshape(-1, 1)).flatten() for feature in features]

# 각 DataFrame에 대한 1/0 레이블 생성
labels = [1,0,0,1,1,1,0,1,1,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,1,0,1,0,1,1,0,1,0,0,0,1,1,0,1,0,0,0,1,0,0,0,1,1,1,0,1,0,0,0,0,0,0,1,0,1,0,1,1,1,1,1,1,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,1,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,1,0,1,1,0,0,1,1,0,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,1,1,0,0,1,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,1,1,0,1,1,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,1,1,0,1,0,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,1,1,1,1,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,1,1,1,0,1,1,0,1,1,0,1,1,0,0,1,0,1,1,1,0,1,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,1,0,0,1,0,1,1,1,0,0,1,0,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,1,1,1,0]

# zero-padding을 적용할 최대 길이 설정 (적절한 값으로 설정해야 함)
max_length = 1000  

# zero-padding 적용
#padded_features = [tf.keras.preprocessing.sequence.pad_sequences([feature.numpy()], maxlen=max_length, padding='post')[0] for feature in features]
padded_features = [tf.keras.preprocessing.sequence.pad_sequences([normalized_features.numpy()], maxlen=max_length, padding='post')[0] for normalized_features in features]

# 데이터셋 생성
dataset = tf.data.Dataset.from_tensor_slices((padded_features, labels))

# 데이터셋을 train 및 validation으로 분리
train_size = int(0.8 * len(padded_features))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)



# Define the TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)

#하이퍼파라미터 정보
hypers = [
    [0.00001, #learning rate
    16,    #batch size
    2000],  #epoch size
    [0.00002, #learning rate
    16,    #batch size
    2000],  #epoch size
    [0.00004, 
    16,    
    2000],
      
    [0.00002, 
    32,    
    2000],  
    [0.00004, 
    32,    
    2000],  
    [0.00006, 
    32,    
    2000],  
    
    [0.00002, 
    100,    
    2000],  
    [0.00004,
    50,
    2000],
    [0.00006,
    50,
    2000],
    [0.00006,
    100,
    2000],
    [0.00006,
    200,
    2000],
    [0.00002,
    100,
    2000],
    [0.00002,
    150,
    2000],
    [0.00002,
    200,
    2000],
    [0.001,
    200,
    2000],
    [0.01,
    200,
    2000],
    [0.1,
    200,
    2000],
    [0.001,
    100,
    2000],
    [0.01,
    100,
    2000],
    [0.1,
    100,
    2000]
]

layerShape = [
    [128,320,16],
    [128,320,32],
    [128,320,64],
    [128,320,128],
    [64,320,16],
    [32,320,32],
    [16,320,64],
    [8,320,128],
    [64,640,16],
    [32,800,32],
    [16,160,64],
    [8,80,128],
]
# 1D CNN 모델 정의


for layer in layerShape:
    # kernel = layer[0]
    # kernel_size = layer[1]
    # dense_size = layer[2]
    for hyper in hypers:
        #epoch
        learning_rate = hyper[0]
        batch_size = hyper[1]
        epoch = hyper[2]
        
        # 데이터셋을 배치로 변환
        train_dataset_local = train_dataset.batch(batch_size)
        val_dataset_local = val_dataset.batch(batch_size)
        
        
        model = Sequential([
        Conv1D(128, 320, activation='relu', input_shape=(max_length, 1)),
        MaxPooling1D(pool_size=6),#그냥 MaxPooling1D로 바꿔보자
        Conv1D(32, 64, activation='relu', input_shape=(max_length, 1)),
        MaxPooling1D(pool_size=3),#그냥 MaxPooling1D로 바꿔보자
        Flatten(),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
        ])

        custom_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        # 모델 컴파일
        model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        # 모델 저장
        
        

        # 모델 요약 확인 (기존))
        #model.summary()
        # 모델 요약 확인 (추가된 것)
        model_summary = []
        model.summary(print_fn=lambda x: model_summary.append(x))
        model_summary = "\n".join(model_summary)
        # 테이블에 저장할 정보
        info_table = pd.DataFrame({
            "Parameter": ["File Prefix", "File Suffix", "Number of Files", "Max Length", "Batch Size", "Learning Rate", "Epochs"],
            "Value": [file_prefix, file_suffix, num_files, max_length, batch_size, custom_optimizer.lr.numpy(), epoch]  
        })

        # 모델 학습
        history = model.fit(train_dataset_local, epochs=epoch, validation_data=val_dataset_local, callbacks=[tensorboard_callback])

        # 결과 테이블에 저장할 정보
        result_table = pd.DataFrame({
            "Metric": ["Train Loss", "Train Accuracy", "Validation Loss", "Validation Accuracy"],
            "Value": [history.history['loss'][-1], history.history['accuracy'][-1], history.history['val_loss'][-1], history.history['val_accuracy'][-1]]
        })

        # 출력
        print("Model Summary:")
        print(model_summary)
        print("\nInformation Table:")
        print(info_table)
        print("\nResult Table:")
        print(result_table)
        
        #파일 출력
        result_table.to_csv(f'result_table_lr{learning_rate}_bs{batch_size}_ep{epoch}.csv', index=False)
        model.save_weights(f"model_weights_lr{learning_rate}_bs{batch_size}_ep{epoch}.hdf5")

        # 모델 평가
        loss, accuracy = model.evaluate(val_dataset_local)
        print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

        # 학습 곡선 시각화
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f"lr{learning_rate}_bs{batch_size}_ep{epoch}")
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()
        plt.close()

